### 一、Introduction to Value Function Approximation

#### 1. Why? (为什么需要函数近似？)

**核心问题：表格法的局限性。**

*   **scalability问题：**
    *   状态空间或动作空间过大时，表格无法存储。例如，Atari游戏图像有 256^84x84 种可能状态，围棋有 10^170$种状态，无法用表格表示。
    *   连续状态空间（如机器人关节角度）有无限多个状态，表格法完全失效。
*   **泛化问题：**
    *   表格法中，每个状态的价值是独立学习的。如果智能体没有访问过某个状态，它就对这个状态一无所知。
    *   现实中，相似的状态应该具有相似的价值。函数近似器可以通过从已见状态中学习到的特征，来**泛化**到未见过但相似的状态。

**结论：为了处理大规模、高维度的RL问题，并让智能体拥有泛化能力，我们必须使用函数近似。**



#### 2. Which Function Approximator?
这是一个由简到繁的演进过程，也是深度学习如何融入RL的脉络。

*   **a 线性模型**
    *   **形式：** $v̂(s, w) = x(s)^T · w$ 或 $q̂(s, a, w) = x(s, a)^T · w$
    *   **特征 $x(s)$：** 核心是特征工程。可以是：
        *   多项式基
        *   傅里叶基
        *   **瓦片编码**：在连续空间中非常高效和流行的选择。
    *   **优点：** 理论收敛性有保证，简单、稳定。
    *   **缺点：** 表征能力有限，依赖好的特征工程。

*   **b 神经网络**
    *   **形式：** $v̂(s, w)$ 或 $q̂(s, a, w)$ 由一个深度神经网络表示。
    *   **优点：**
        *   **强大的表征能力**：无需复杂的特征工程，能从原始输入（如像素）中自动学习高级特征。
        *   **端到端学习**。
    *   **缺点：**
        *   **理论收敛性保证弱**，训练可能不稳定。
        *   **需要大量数据**。
        *   **容易过拟合**。

**现代深度RL的默认选择是神经网络。**

---

### 二、How? (如何训练函数近似器？)

这部分是你的笔记框架可以扩展的**核心**，它回答了“我们怎么教这个网络学会正确的价值”。

#### 1. 基本思路：基于价值函数的目标

我们有一个目标 $y$，希望模型的预测 $v̂(s, w)$ 尽可能接近这个目标。

*   **对于 MC 方法：** 目标 $y = G_t$（实际回报）。我们使用监督学习，最小化 $[G_t - v̂(S_t, w)]^2$。
*   **对于 TD(0) 方法：** 目标 $y = R_{t+1} + γv̂(S_{t+1}, w)$。最小化 $[R_{t+1} + γv̂(S_{t+1}, w) - v̂(S_t, w)]^2$。
    *   **注意：** 这里的目标 $y$ 本身依赖于当前的网络参数 $w$，这引入了偏差，但减少了方差。

#### 2. 核心算法演进

*   **a) DQN (Deep Q-Network)**
    *   **贡献1：经验回放**
        *   **是什么：** 将智能体的经历 $(s, a, r, s')$ 存储在一个固定大小的回放缓冲区中。训练时，从缓冲区中**随机采样**一个batch的数据。
        *   **为什么：**
            1.  **打破数据间的相关性**：连续的状态序列是高度相关的，随机采样破坏了这种相关性，使训练更稳定。
            2.  **数据重用**：提高样本效率。
    *   **贡献2：目标网络**
        *   **是什么：** 使用一个独立的网络（目标网络）来计算TD目标 $y = r + γ * max_{a'} Q(s', a'; w^-)$。这个网络的参数 $w^-$ 每隔一定步数才从主网络 $w$ 同步一次。
        *   **为什么：** 解决“移动目标”问题。如果使用同一个网络计算目标和预测，目标会随着预测的更新而不断变化，导致训练振荡和发散。目标网络提供了一个相对稳定的目标。

*   **b) 对DQN的改进**
    *   **Double DQN：** 解决DQN对Q值**过估计**的问题。解耦“动作选择”和“价值评估”，使Q值估计更准确。
    *   **Dueling DQN：** 改变网络架构，将 $Q(s, a)$ 分解为 **状态价值 $V(s)$** 和 **优势函数 $A(s, a)$** 之和。网络能更高效地学习哪些状态是好的，而不关心每个动作细微的差异。
    *   **Prioritized Experience Replay：** 不是均匀地从回放缓冲区采样，而是优先采样那些“TD误差”大的经历，因为这些经历能带来更大的学习价值。

---

### 三、Challenges & Pitfalls (挑战与陷阱)

这是从理论到实践的关键，也是CS285非常强调的部分。

1.  **训练不稳定性**
    *   **原因：**
        *   数据非独立同分布
        *   目标值依赖于网络自身（Bootstrapping）
        *   函数近似器的引入使得策略的微小变化可能导致价值的巨大跳跃
    *   **解决方案：** 经验回放、目标网络、**梯度裁剪**、谨慎调整学习率。

2.  **灾难性遗忘**
    *   **现象：** 网络在学习新知识时，迅速忘记了旧知识。
    *   **解决方案：** 经验回放（混入旧数据）、持续学习技术。

3.  **过估计**
    *   **现象：** 估计的Q值系统地高于真实值。
    *   **解决方案：** Double DQN。



### 二、incremental methods

#### 1. 核心假设：线性模型
$$\hat{v}(S, w) = x(S)^T w = \sum_{j=1}^n x_j(S)w_j$$

*   **这是什么？** 这是我们用来近似真实价值函数 $v_π(S)$ 的模型。
*   **$x(S)$：状态 $S$ 的特征向量。** 这是关键！我们不再用表格记录每个状态，而是用一组特征来描述一个状态。例如：
    *   在赛车游戏中，特征可以是：$[位置, 速度, 剩余燃油, 与对手的距离]$。
    *   这实现了**泛化**：两个特征相似的状态，会被模型赋予相似的价值。
*   **$w$：权重参数。** 这是我们需要通过学习来调整的。每个权重 $w_j$ 代表了其对应特征 $x_j(S)$ 的重要性。
*   **$x(S)^T w$：线性组合。** 模型的预测值是特征和权重的点积。这是一个线性模型。

#### 2. 目标：最小化误差
$$J(w) = E_\pi \left[ (v_\pi(S) - x(S)^T w)^2 \right]$$

*   **$J(w)$：目标函数（损失函数）。** 我们的目标是找到一组权重 $w$，使得这个 $J(w)$ 最小。
*   **$(v_π(S) - x(S)^T w)^2$：预测误差的平方。** 我们希望模型的预测值 $x(S)^T w$ 尽可能接近真实的价值 $v_π(S)$。平方是为了保证误差为正且便于求导。
*   **$E_π[...]$：关于状态分布的期望。** 我们不是只关心某一个状态，而是关心在所有状态上（按照策略 $π$ 访问它们的频率）的平均表现。

#### 3. 算法的优势：收敛性与简洁性
*   **“Stochastic gradient descent converges on global optimum”**
    *   **为什么？** 因为损失函数 $J(w)$ 是参数 $w$ 的**二次函数**。二次函数只有一个“碗”状的结构（一个全局最优点），没有复杂的局部最优点。
    *   因此，即使是随机梯度下降这种有噪声的算法，也**保证能收敛**到全局最优解。这是线性模型一个非常强大的理论保证，在复杂的非线性模型（如神经网络）中是不存在的。

#### 4. 更新规则：直观且高效
$$\nabla_w \hat{v}(S, w) = x(S)$$
$$\Delta w = \alpha(v_\pi(S) - \hat{v}(S, w))x(S)$$

这是幻灯片最核心的部分，我们来一步步拆解：

*   **第一步：计算梯度。**
    *   梯度 $∇_w v̂(S, w)$ 告诉我们，为了增加 $v̂$ 的输出，我们应该如何调整每个权重 $w_j$。
    *   对于线性模型，这个梯度非常简单，就是**特征向量 $x(S)$ 本身**。因为 $∂(x(S)^T w) / ∂w_j = x_j(S)$。
    *   **直觉：** 每个权重的梯度大小，直接等于其对应的特征值。特征值越大，说明这个特征在当前状态下越“显著”，那么它的权重就应该得到越大的调整。

*   **第二步：应用梯度下降更新。**
    *   $α$：学习率（步长），控制更新幅度。
    *   $(v_π(S) - v̂(S, w))$：**预测误差（TD Error的雏形）**。这是驱动学习的根本信号。
        *   如果预测低了 ($v_π(S) > v̂$)，误差为正，我们会增加 $w$，从而提高对这个状态的预测。
        *   如果预测高了 ($v_π(S) < v̂$)，误差为负，我们会减少 $w$，从而降低预测。
    *   $x(S)$：特征向量。它决定了**如何分配这个误差**。
        *   只有那些在当前状态 $S$ 下**非零的特征**所对应的权重才会被更新。
        *   如果一个特征 $x_j(S) = 0$，那么 $Δw_j = 0$，它的权重就不会改变。

#### 5. 精髓总结
$Update = step-size × prediction error × feature value$

这个总结非常精辟。它把复杂的数学公式变成了一个直观的三因子乘法：
*   **$step-size$ (α):** 你学习的“激进”程度。
*   **$prediction error$ (δ):** 你这次“错得有多离谱”，这是学习的方向和动力。
*   **$feature value$ (x(S)):** 哪个“线索”应该为这次错误“负责”，就在哪个方向上修正。

### 从理论到现实的挑战

幻灯片描述的是一个**理想情况**，它基于我们之前讨论的 **“Oracle”**——即我们知道每个状态的**真实价值 $v_π(S)$**。

在现实中，我们**不知道 $v_π(S)$**。因此，我们必须用一个**目标**来替代它，这就引出了不同的算法：

*   **对于蒙特卡洛方法：**
    *   用**实际回报 $G_t$** 作为 $v_π(S)$ 的替代。
    *   更新规则变为：$Δw = α(G_t - v̂(S_t, w)) x(S_t)$
    *   这是**无偏**的，但**方差高**。

*   **对于时序差分方法（如TD(0)）：**
    *   用**TD目标 $R_{t+1} + γv̂(S_{t+1}, w)$** 作为 $v_π(S)$ 的替代。
    *   更新规则变为：$Δw = α( [R_{t+1} + γv̂(S_{t+1}, w)] - v̂(S_t, w) ) x(S_t)$
    *   这就是**线性函数近似下的TD(0)**。它引入了**偏差**，但大大降低了**方差**。


让我们以Windy GridWorld为例，把线性函数近似和资格迹结合起来。

### 第一步：回顾问题 - 没有资格迹时的问题是什么？

在普通的线性TD(0)中，我们有一条经验 `(s, a, r, s')`，然后我们更新权重：
`Δw = α * (r + γ*V(s') - V(s)) * x(s)`

**这里的关键是：我们只根据当前状态的特征 `x(s)` 来更新权重。**

这在Windy GridWorld中会导致什么问题？
想象一下：你从起点开始，经过状态 `s1, s2, s3, ...`，最后到达终点获得一个大奖励。TD(0)只会将最终的成功**直接归因**给最后一步的状态 `s_{final}`。`s_{final}` 的特征 `x(s_final)` 会得到更新，但之前所有导致这个成功的关键状态（比如在风中艰难保持位置的状态）却**得不到应有的功劳**，因为当你访问它们时，奖励还没有发生。

这就是**信用分配问题**：如何将最终的奖励/成功，合理地分配给之前所有导致这一结果的状态？

---

### 第二步：资格迹的直观理解 - “痕迹”与“足迹”

资格迹引入了一个非常巧妙的机制来解决信用分配问题。

**核心比喻：足迹和潮湿的地面**

想象智能体在潮湿的地面上行走。每访问一个状态 `s`，它就在地上留下一个“泥脚印”。这个脚印会随着时间慢慢变干（衰减）。

*   **当一个奖励发生时**，我们不仅更新当前状态，**还会去更新所有还有“泥印”的状态**。脚印越深、越新鲜的状态，获得的更新就越大。
*   这样，最终获得的大奖励，就能**沿着智能体来时的路径，反向传播回去**，分配给所有相关的状态。

**在数学上，这就是资格迹向量 `e(t)`**。它为**每个权重** `w_i` 都维护了一个“痕迹”。

---

### 第三步：线性TD(λ)的具体机制

现在我们把资格迹融入到线性函数近似中。

#### 1. 初始化
*   权重向量 `w` 随机初始化。
*   资格迹向量 `e` 初始化为全零（地面一开始是干的）。

#### 2. 每一步的更新（对每个时间步t）

对于每一步 `(s, a, r, s')`：

**a) 计算TD误差：**
`δ = r + γ * V(s') - V(s)`
其中 `V(s) = x(s)^T * w`，`V(s') = x(s')^T * w`

**b) 更新资格迹：**
`e = γ * λ * e + x(s)`
让我们拆解这个公式：
*   `x(s)`：**留下新脚印**。你当前访问的状态 `s`，它的特征向量 `x(s)` 给地面留下了新的泥印。
*   `γ * λ * e`：**旧脚印变干**。所有旧的痕迹都衰减为原来的 `γ * λ` 倍。
    *   `γ`（折扣因子）：未来的奖励不如眼前的奖励重要，所以未来的状态痕迹也应该衰减得更快。
    *   `λ`（迹衰减因子）：`λ=0` 表示立刻忘记之前的状态（退化为TD(0)）；`λ=1` 表示非常长的记忆。

**c) 更新权重：**
`Δw = α * δ * e`
`w = w + Δw`

**这是最关键的一步！** 注意，我们是用**整个资格迹向量 `e`** 来更新权重，而不仅仅是当前状态的特征 `x(s)`。

这意味着，当前计算出的TD误差 `δ`，会用来更新**所有在过去被访问过且痕迹尚未完全消失的状态**。`e` 向量中的每个元素 `e[i]` 记录了特征 `i` 在最近的历史中出现的频率和新鲜度。

---

### 第四步：在Windy GridWorld中的具体过程

假设你的特征 `x(s)` 是位置编码（比如所在行列的one-hot编码）。
1.  你从起点 `S0` 开始。`e` 被 `x(S0)` 填充。`e` 现在记录了起点。
2.  你逆风移动了一步，到了 `S1`，没得到奖励（`r=0`）。TD误差 `δ` 很小。
    *   首先，`e` 衰减然后加上 `x(S1)`。
    *   然后用这个 `δ` 和 `e` 去更新 `w`。因为 `e` 里既有 `S1` 的痕迹，也有 `S0` 的痕迹，所以**这两个状态的估值都会根据这次平凡的转移进行微调**。
3.  你继续艰难地在风中移动 `S2, S3, ...`，每一步都在 `e` 中累积你的路径痕迹，并不断地用小的TD误差微调沿路的所有状态。
4.  终于，你到达终点 `S_goal`，获得了大奖励 `r=+10`。
    *   此时计算出的TD误差 `δ` 会很大（因为获得了意想不到的大奖励）。
    *   此时的资格迹向量 `e` 包含了从起点 `S0` 到终点 `S_goal` 的**整条路径的衰减痕迹**！
    *   执行 `w = w + α * δ * e`。这个巨大的、正的TD误差 `δ` 将乘以 `e`，然后**加到你整条路径上所有状态的权重上**。

**结果就是：** 导致最终成功的**整条路径**上所有状态的估值都得到了提升，而不仅仅是终点状态。离终点越近的状态，痕迹越新鲜（在 `e` 中占比可能越高），获得的提升也越大。

### 总结

*   **资格迹 `e`** 是一个**短期记忆向量**，记录了哪些特征（对应的权重）在最近被“激活”过，以及它们激活的“新鲜度”。
*   **TD(λ)** 的核心思想是：**当前的TD误差不仅用于更新当前状态，还用于更新所有带有“痕迹”的过去状态。**
*   **λ** 控制着这个记忆的长度：`λ=0`（健忘症，只记当前状态），`λ=1`（永久记忆，但被γ衰减）。

通过这个机制，线性TD(λ) 高效地解决了信用分配问题，使得学习过程更加平滑和高效。在像Windy GridWorld这样奖励稀疏且延迟的环境中，它的优势会非常明显。



### 问题：我又有一点混乱，比如我们使用td方法来学习gridworld，我们不是本来要学习Q值吗，那这样的话什么时候进行以上这个近似函数的学习呢

总结：解决你的混乱

*   **你的混淆点在于：** 你以为学习Q值是一套方法，而函数近似是另一套独立的方法。
*   **正确的认知是：** 函数近似（包括你幻灯片上的线性模型）是 **【表示Q值/价值的一种方式】**，而TD学习是 **【学习这种表示形式的算法】**。

**在表格法中：**
*   Q值被直接存储在 `Q[s][a]` 这个表格里。
*   TD更新是直接修改表格中的某个格子：`Q[s][a] = Q[s][a] + α * (TD_error)`

**在函数近似中：**
*   Q值由一个参数化函数 `ˆq(s, a, w)` 表示。
*   TD更新是修改这个函数的**参数 `w`**：`w = w + α * (TD_error) * ∇_w ˆq(s, a, w)`
*   对于线性函数，`∇_w ˆq(s, a, w)` 就是 `x(s, a)`。

所以，**你一直都在进行“近似函数的学习”**。从你决定用 `ˆq(s, a, w)` 来代替Q表格的那一刻起，你每一步的TD更新都是在调整这个近似函数的参数 `w`，让它产生的Q值越来越准。




### 三、Batch Methods（批处理方法）

#### 1. Motivation（动机）

**核心问题：增量法的低数据效率与不稳定性**

我们之前学的TD(0)、Q-learning、Sarsa都是**增量法**。它们通过**单步经验** (s, a, r, s') 来立即更新价值函数或策略。

这存在两个主要缺点：
*   **数据效率低下：** 每个经验(s, a, r, s') 只使用一次就被丢弃了。
*   **训练不稳定：**
    *   连续到达的经验之间通常是**高度相关**的（比如在GridWorld中，你一步接一步地走）。这违反了许多收敛定理所要求的“独立同分布”假设。
    *   当前的价值函数估计会立即影响下一个TD目标，导致学习过程像在追逐一个**移动的目标**，容易产生振荡或发散。

**Batch Methods的解决方案：**
**不再逐条处理经验，而是收集一批经验数据（一个“Batch”），然后利用这整批数据来寻找一个“最优”的价值函数或策略，使其能够最好地“拟合”这批观测到的数据。**

**一个生动的比喻：**
*   **增量法（如TD(0)）：** 像一个小学生，每做一道题就立刻对答案，然后根据对错稍微调整自己的思路。
*   **批处理方法：** 像一个科学家，先进行一系列实验（收集一个Batch的数据），然后坐下来，分析所有实验数据，从中找出一个最能解释所有实验结果的理论模型。

#### 2. Least Squares Prediction（最小二乘预测）

这是批处理方法中最基础、最直观的一种，用于**价值函数预测**。

**设定：**
*   我们有一批（固定不变的）经历，形式为 (s, v_π(s))。其中 v_π(s) 是状态 s 的**真实价值**。
*   我们使用线性函数近似：v̂(s, w) = x(s)^T w。
*   我们的目标是找到权重 w，使得对于这批数据中的所有状态，预测值 v̂(s, w) 和真实值 v_π(s) 的**均方误差最小**。

**数学形式：**
假设我们有 m 个状态，其真实价值构成向量 V，其特征矩阵为 X（第i行是状态 s_i 的特征 x(s_i)^T）。
目标是最小化平方和损失：
J(w) = Σ_i (v_π(s_i) - x(s_i)^T w)^2 = || V - Xw ||^2

**解：**
这个经典的线性回归问题有**解析解**（闭合形式的解），称为**最小二乘解**：
w = (X^T X)^{-1} X^T V

**然而，在RL中我们面临一个核心挑战：**
**我们并不知道真实价值 v_π(s)！** 这就是引入“Oracle”概念的地方。

**解决方法：用不同的“目标”来替代 v_π(s)：**

*   **如果使用蒙特卡洛目标：** 用实际回报 G_t 替代 v_π(s)。这就变成了 **Least Squares Monte Carlo (LSMC)**。
*   **如果使用时序差分目标：** 用TD目标替代 v_π(s)。例如，对于TD(0)，目标就是 r + γv̂(s', w)。但这里有个问题：目标 r + γv̂(s', w) **依赖于我们正在寻找的 w**！这就引出了...

#### 3. Stochastic Gradient Descent with Experience Replay（带经验回放的随机梯度下降）

这是连接传统批处理方法和现代深度Q网络（DQN）的**关键桥梁**。

**a) 核心思想：**
1.  **存储：** 将智能体与环境交互的经验 (s, a, r, s') 存储在一个固定大小的**经验回放缓冲区** 中。
2.  **重复利用：** 在训练时，**随机地从缓冲区中采样一小批** 经历。
3.  **学习：** 对这一小批经历，像普通的SGD一样执行梯度下降更新。

**b) 为什么有效？**
*   **打破相关性：** 随机采样打破了连续经验之间的时序相关性，使得数据更接近“独立同分布”，大大提高了训练的稳定性。
*   **数据重用：** 每一条经验都可以被多次用于学习，极大地提高了数据效率。

**c) 与最小二乘法的关系：**
**经验回放SGD实际上是在迭代地、随机地求解最小二乘问题。**
*   最小二乘法是**一次性**使用**所有**数据，计算出最优解 w*。
*   经验回放SGD是**重复、随机地**使用**数据子集**，一步步地**逼近**最优解 w*。

**在价值函数近似中的具体算法（DQN的前身）：**

对于线性函数 q̂(s, a, w)，给定一批经验 (s, a, r, s')：

1.  对于经验中的每个样本，计算Q-learning目标：
    $y_j = r_j + γ * max_{a'} q̂(s'_j, a', w)$
    （注意：这里的目标依赖于当前的 w，目标在移动）

2.  计算这批数据上的损失（如均方误差）：
    $L(w) = Σ_j (y_j - q̂(s_j, a_j, w))^2$

3.  执行梯度下降更新：
    $w ← w - α * ∇_w L(w)$

**d) 进化到DQN：**
DQN在此基础上增加了两个关键创新，进一步稳定了训练：
1.  **目标网络：** 使用一个独立的、更新缓慢的目标网络来计算TD目标 y_j，解决了“移动目标”问题。
2.  **经验回放：** 直接继承自本节的SGD with Experience Replay。

### 总结

*   **动机：** 解决增量法数据效率低和不稳定的问题。
*   **Least Squares Prediction：** 展示了批处理方法的**最优思想**——寻找最能拟合所有数据的模型。它提供了理论基准。
*   **SGD with Experience Replay：** 提供了**实现批处理思想的实用算法**。它通过迭代和随机采样，用计算上的高效换取了数据上的高效和训练的稳定，是通往现代深度RL的基石。

### DQN
完美！这张幻灯片精准地概括了Deep Q-Network (DQN) 之所以能成功稳定训练的两个最关键的创新。它就是你之前学习的 **“Batch Methods”** 和 **“SGD with Experience Replay”** 思想的直接应用和升华。


#### 核心机制：两大稳定器

**1. Experience Replay（经验回放）**
*   **“Store transition ... in replay memory D” & “Sample random mini-batch ... from D”**
*   **这是什么？** 这就是 **“Stochastic Gradient Descent with Experience Replay”** 的实践！
*   **为什么这么做？**
    *   **打破相关性：** 连续的状态 (s_t, s_{t+1}, ...) 是高度相关的。直接从连续序列中学习，就像试图从一本情节连贯的小说里随机挑句子来理解语法，会很困难。经验回放通过**随机采样**，把连续的剧情打乱成独立的“事实”，使数据分布更平滑，大大提升了训练的稳定性。
    *   **数据重用：** 每一条经验都可以被多次学习，极大地提升了数据效率。

**2. Fixed Q-Targets（固定Q目标）**
*   **“Compute Q-learning targets w.r.t. old, fixed parameters w⁻”**
*   **这是什么？** 这是为了解决 **“移动目标”** 问题。
*   **问题所在：** 在原始的Q-learning中，目标 y = r + γ * max_{a'} Q(s', a'; w) 依赖于当前的参数 w。而 w 是每步都在更新的。这就导致目标 y 就像一个**移动的靶心**，你在瞄准的同时，靶子也在跑，导致训练振荡甚至发散。
*   **解决方案：** DQN引入了**目标网络**，其参数 w⁻ 是从主网络**定期复制**过来的（例如，每1000步同步一次）。在很长一段时间内，目标 y = r + γ * max_{a'} Q(s', a'; w⁻) 是**固定不变的**。这就提供了一个稳定的目标，让主网络可以可靠地朝着它优化。这就像在射击训练中，先固定住靶子，让你练习瞄准。

#### 算法流程分解

1.  **Take action a_t according to ϵ-greedy policy**
    *   **平衡探索与利用。** 这是保证能收集到有价值数据的前提。

2.  **Store transition $(s_t, a_t, r_{t+1}, s_{t+1})$ in replay memory D**
    *   **积累数据。** 构建我们的“Batch”数据集。

3.  **Sample random mini-batch of transitions (s, a, r, s') from D**
    *   **执行经验回放。** 从历史数据中随机抽取一个小批量，用于本次更新。

4.  **Compute Q-learning targets w.r.t. old, fixed parameters w⁻**
    *   **计算稳定目标。** 使用**目标网络**计算 $y = r + γ * max_{a'} Q(s', a'; w⁻)$。

5.  **Optimise MSE between Q-network and Q-learning targets**
    *   **执行梯度下降更新。** 让主网络的预测 Q(s, a; w) 去逼近我们计算出的稳定目标 y。

#### 损失函数详解

$$L_i(w_i) = E_{s,a,r,s' \sim D_i} \left[ \left( \underbrace{r + \gamma \max_{a'} Q(s', a'; w_i^-)}_{\text{Fixed Target } y} - Q(s, a; w_i) \right)^2 \right]$$

*   **$L_i(w_i)$**： 在第 i 次迭代时，关于主网络参数 w_i 的损失函数。
*   **$E_{s,a,r,s' ~ D_i}$**： 期望项，表示我们对从经验回放缓冲区 D 中采样的小批量数据求平均。
*   **$r + γ * max_{a'} Q(s', a'; w_i⁻)$**： 这就是 **“固定的Q目标”** y。它是我们希望Q网络去逼近的目标值。
    *   w_i⁻ 是**目标网络**的参数，在迭代过程中是**固定**的。
*   **$Q(s, a; w_i)$**： 这是**主网络**对当前状态-动作对的估值。
*   **整个损失**： 就是标准的**均方误差**，衡量了主网络的预测与其**稳定目标**之间的差距。

### 总结：DQN的突破性贡献

这张幻灯片展示的DQN，其伟大之处在于它**巧妙地组合了已有的思想，解决了深度神经网络在RL中应用时的核心稳定性问题**。

*   它用 **Experience Replay** 解决了**数据相关性和效率**问题。
*   它用 **Fixed Q-Targets** 解决了**目标不稳定**问题。

这两大支柱，使得用复杂的非线性函数近似器（深度神经网络）来学习Q函数成为可能，从而开启了深度强化学习的新时代。你之后看到的所有更先进的算法（如DDPG, Rainbow等），几乎都建立在DQN的这些核心思想之上。